# Load and strip the original data

In [1]:
import pandas as pd
from os.path import join
from bcrypt import gensalt, hashpw

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

In [2]:
src = "../../data/original/Grunddaten-Simulation"
dst = "../../data/raw/"

In [3]:
def hash_id(ID):
    hashed_id = hashpw(str(ID).encode('utf-8'), salt=salt)
    return hashed_id[29:].decode()

In [3]:
# for hashing the student and lecturer IDs
salt = gensalt(12)

In [4]:
salt

b'$2b$12$moTL.FsFqoZnhSFv7vjkme'

## Students

File `Studiendaten.csv`, sample stored in `studies_P.csv`
* `ST_PERSON_NR`: rename to `student_id`, hash
* `STUDIENIDENTIFIKATOR`: rename to `study_id`
* `STUDIENBEZEICHNUNG`: rename to `study_name`
* `SEMESTERANZAHL`: rename to `term_number`

In [38]:
# List of studies for every student. A student can have more than one study,
# which will show up as separate entries (row) for the same student_id. Each
# study also has a term number, i.e. the number of semesters the student has
# been enrolled in the given study.
df = pd.read_csv(join(src, 'Studiendaten.csv'), encoding='latin_1')
df = df.rename(columns={
    'ST_PERSON_NR':'student_id', # unique student identifier
    'STUDIENIDENTIFIKATOR':'study_id', # unique study identifier
    'STUDIENBEZEICHNUNG':'study_name', # (german) name of the study
    'SEMESTERANZAHL':'term_number' # number of terms a student has been enrolled
})

# hash student IDs with the given salt
hashed_IDs = []
pool = Pool(12)
for hashed_ID in tqdm(
        pool.imap_unordered(func=hash_id, iterable=df["student_id"]),
        total=len(df["student_id"])
    ):
    hashed_IDs.append(hashed_ID)
    
df["student_id"] = hashed_IDs   
df.to_csv(join(dst, "students.csv"), index=False)

100%|██████████| 24475/24475 [10:14<00:00, 39.80it/s]


## Lecturers

File `Bedienstete_mit_DV_an_Org.csv` (Organisationseinheiten), sample stored in `organisations_P.csv`
* `PERSON_NR`: rename to `lecturer_id`, hash
* `ORG_NR`: drop
* `TUG_NEW.PUORG.GETNAME(A.ORG_NR)`: rename to `organisation_name`

In [36]:
# Mapping of lecturers to organisations (institute, faculty). A lecturer can
# be associated with more than one organisation.
df = pd.read_csv(join(src, 'Bedienstete_mit_DV_an_Org.csv'),
                            encoding='latin_1')
df = df.rename(columns={
    'PERSON_NR':'lecturer_id', # unique lecturer id
    'TUG_NEW.PUORG.GETNAME(A.ORG_NR)':'organisation_name' # German organisation name
})

df = df.drop(columns=[
    "ORG_NR"
])

# hash lecturer IDs with the given salt
hashed_IDs = []
pool = Pool(12)
for hashed_ID in tqdm(
        pool.imap_unordered(func=hash_id, iterable=df["lecturer_id"]),
        total=len(df["lecturer_id"])
    ):
    hashed_IDs.append(hashed_ID)
    
df["lecturer_id"] = hashed_IDs
df.to_csv(join(dst, "lecturers.csv"), index=False)

100%|██████████| 5707/5707 [02:22<00:00, 40.13it/s]


## Courses

File `LV_cleaned.csv`, sample stored in `courses.csv`
* `STP_SP_NR`: rename to `lecture_id`
* `STP_SP_LVNR`: drop
* `SJ_NAME`: drop
* `SEMESTER_KB`: drop
* `STP_SP_TITEL`: drop
* `STP_SP_TITEL_ENGL`: rename to `course_name`
* `STP_SP_SST`: drop
* `STP_LV_ART_KURZ`: rename to `course_type`, provide dictionary
* `STP_LV_ART_NAME`: drop
* `BETREUENDE_ORG_NR`: drop
* `BETREUENDE_ORG_NAME`: drop

In [70]:
# list of lectures was manually cleaned, since for some rows, the entries starting
# from the column STP_LV_ART_KURZ were shifted to the right by one column

# list of lectures with information about their type, their name, their module
# (this is only relevant for how studies are composed at TU Graz) and the 
# organisational unit (institute, faculty) which is responsible for the lecture.
df = pd.read_csv(join('../../data/cleaned', 'LV_cleaned.csv'), encoding="utf-8")
df = df.rename(columns={
    'STP_SP_NR':'course_id', # unique course id
    'STP_SP_TITEL_ENGL':'course_name', # english lecture name
    'STP_LV_ART_KURZ':'course_type', # type of the lecture (tutorial, lab, ...)
})
df = df.drop(columns=[
    "SJ_NAME",
    "SEMESTER_KB",
    "STP_SP_SST",
    "STP_LV_ART_NAME",
    "STP_SP_TITEL",
    'STP_SP_LVNR',
    'BETREUENDE_ORG_NR',
    'BETREUENDE_ORG_NAME',
    "Unnamed: 11"
])
df.to_csv(join(dst, "courses.csv"), index=False)

## Course enrollment by students

File `Studierende_pro_LV_mit_Idf.csv`, sample stored in `course_enrollment.csv`
* `ST_PERSON_NR`: rename to `student_id`, hash
* `STUDIENIDENTIFIKATOR`: rename to `study_id`
* `STP_SP_NR`: rename to `course_id`
* `LV_GRP_NR`: rename to `group_id`

In [37]:
# List of enrolled lectures of the WiSe 2019/20 for every student. A lecture
# can have several groups (for example for tutorial parts). The group
# identifier is also listed for every student. It is not completely unique
# as there are a number of overlapping groups (for example same time, 
# different rooms). These are disambiguated at a later point in the data
# cleaning process.
# The data also includes the identifier of the study through which the 
# student enrolled in a given lecture. 
df = pd.read_csv(join(src, 'Studierende_pro_LV_mit_Idf.csv'))
df = df.rename(columns={
    'ST_PERSON_NR':'student_id', # unique student identifier
    'STUDIENIDENTIFIKATOR':'study_id', # unique study identifier
    'STP_SP_NR':'course_id', # unique course identifier
    'LV_GRP_NR':'group_id', # (almost) unique group identifier 
    }) 

# hash student IDs with the given salt
hashed_IDs = []
pool = Pool(12)
for hashed_ID in tqdm(
        pool.imap_unordered(func=hash_id, iterable=df["student_id"]),
        total=len(df["student_id"])
    ):
    hashed_IDs.append(hashed_ID)
    
df["student_id"] = hashed_IDs   
df.to_csv(join(dst, "course_enrollment.csv"), index=False)

100%|██████████| 85505/85505 [36:15<00:00, 39.31it/s]


## Exam enrollment by students

File `Prüfungen-2.csv`, sample stored in `exam_students_P.csv`: 
* `PV_TERM_NR`: rename to `exam_id`
* `PRUEFUNGSDATUM`: drop
* `ST_PERSON_NR`: rename to `student_id`, hash
* `STUDIENIDENTIFIKATOR`: rename to `study_id`
* `STP_SP_NR`: rename to `course_id`

In [6]:
df = pd.read_csv(join(src, 'Prüfungen-2.csv'), encoding='latin_1')
df = df.rename(columns={
    'PV_TERM_NR':'exam_id',
    'ST_PERSON_NR':'student_id',
    'STUDIENIDENTIFIKATOR':'study_id',
    'STP_SP_NR':'course_id'})

df = df.drop(columns=[
    'PRUEFUNGSDATUM'
])

# hash student IDs with the given salt
hashed_IDs = []
pool = Pool(12)
for hashed_ID in tqdm(
        pool.imap_unordered(func=hash_id, iterable=df["student_id"]),
        total=len(df["student_id"])
    ):
    hashed_IDs.append(hashed_ID)

df["student_id"] = hashed_IDs
df.to_csv(join(dst, "exam_enrollment.csv"), index=False)

100%|██████████| 57815/57815 [24:12<00:00, 39.79it/s]


## Course supervision

File `Lehrende.csv`, sample stored in `lecturers_P.csv`
* `PERSON_NR`: rename to `lecturer_id`, hash
* `STP_SP_NR`: rename to `course_id`
* `LV_GRP_NR`: rename to `group_id`

In [ ]:
# list of lecturers which are responsible for lectures and groups within
# lectures. Similar to the list of students, the group_id is disambiguated
# later in the data cleaning process
df = pd.read_csv(join(src, 'Lehrende.csv'))
df = df.rename(columns={
    'PERSON_NR':'lecturer_id', # unique lecturer id
    'STP_SP_NR':'course_id', # unique lecture id
    'LV_GRP_NR':'group_id'# (almost) unique group id
})

# hash lecturer IDs with the given salt
hashed_IDs = []
pool = Pool(12)
for hashed_ID in tqdm(
        pool.imap_unordered(func=hash_id, iterable=df["lecturer_id"]),
        total=len(df["lecturer_id"])
    ):
    hashed_IDs.append(hashed_ID)
    
df["lecturer_id"] = hashed_IDs
df.to_csv(join(dst, "course_supervision.csv"), index=False)

## Exam supervision

File `Prüfungstermine_mit_Räumen.csv`, sample stored in `exam_dates_P.csv`:
* `PV_TERM_NR`: rename to `exam_id`
* `PERSON_NR`: rename to `lecturer_id`, hash
* `DATUM`: drop
* `BEGINNZEIT`: drop
* `ENDEZEIT`: drop
* `RES_NR`: drop
* `STP_SP_NR`: rename to `course_id`

In [23]:
df = pd.read_csv(join(src, 'Prüfungstermine_mit_Räumen.csv'), encoding='latin_1',
                     parse_dates=['DATUM'], dayfirst=True)
df = df.rename(columns={
    'PV_TERM_NR':'exam_id',
    'PERSON_NR':'lecturer_id',
    'STP_SP_NR':'course_id'})

df = df.drop(columns=[
    'DATUM',
    'BEGINNZEIT',
    'ENDEZEIT',
    'RES_NR',
])

# hash lecturer IDs with the given salt
hashed_IDs = []
pool = Pool(12)
for hashed_ID in tqdm(
        pool.imap_unordered(func=hash_id, iterable=df["lecturer_id"]),
        total=len(df["lecturer_id"])
    ):
    hashed_IDs.append(hashed_ID)
    
df["lecturer_id"] = hashed_IDs
df.to_csv(join(dst, "exam_supervision.csv"), index=False)

100%|██████████| 5633/5633 [02:22<00:00, 39.63it/s]


## Course dates and rooms

File `Termine_mit_LV Bezug.csv`, sample stored in `dates.csv`
* `RES_NR`: rename to `room_id`
* `DATUM_AM`: rename to `date`
* `ZEIT_VON`: rename to `start_time`
* `ZEIT_BIS`: rename to `end_time`
* `STP_SP_NR`: rename to `course_id`
* `LV_GRP_NR`: rename to `group_id`

In [67]:
# events (start time, end time, room) for every lecture and group in WiSe 2019/20
df = pd.read_csv(join(src, 'Termine_mit_LV Bezug.csv'),
                parse_dates=['DATUM_AM', 'ZEIT_VON', 'ZEIT_BIS'], dayfirst=True)
df = df.rename(columns={
    'RES_NR':'room_id', # unique room id
    'DATUM_AM':'date', # date
    'ZEIT_VON':'start_time', # start time
    'ZEIT_BIS':'end_time', # end time
    'STP_SP_NR':'course_id', # unique lecture id
    'LV_GRP_NR':'group_id'# (almost) unique group id
})
df.to_csv(join(dst, "course_dates.csv"), index=False)

## Exam dates and rooms

File `Prüfungstermine_mit_Räumen.csv`, sample stored in `exam_dates_P.csv`:
* `PV_TERM_NR`: rename to `exam_id`
* `PERSON_NR`: drop
* `DATUM`: rename to `date`
* `BEGINNZEIT`: rename to `start_time`
* `ENDEZEIT`: rename to `end_time`
* `RES_NR`: rename to `room_id`
* `STP_SP_NR`: rename to `course_id`

In [23]:
df = pd.read_csv(join(src, 'Prüfungstermine_mit_Räumen.csv'), encoding='latin_1',
                     parse_dates=['DATUM'], dayfirst=True)
df = df.rename(columns={
    'PV_TERM_NR':'exam_id',
    'DATUM':'date',
    'BEGINNZEIT':'start_time',
    'ENDEZEIT':'end_time',
    'RES_NR':'room_id',
    'STP_SP_NR':'course_id'})

df = df.drop(columns=[
    'PERSON_NR',
])

# hash lecturer IDs with the given salt
hashed_IDs = []
pool = Pool(12)
for hashed_ID in tqdm(
        pool.imap_unordered(func=hash_id, iterable=df["lecturer_id"]),
        total=len(df["lecturer_id"])
    ):
    hashed_IDs.append(hashed_ID)
    
df["lecturer_id"] = hashed_IDs
df.to_csv(join(dst, "exam_dates.csv"), index=False)

100%|██████████| 5633/5633 [02:22<00:00, 39.63it/s]


## Rooms

File `Räume_cleaned.csv`, sample stored in `rooms.csv`
* `RES_NR`: rename to `room_id`
* `RAUM_CODE`: drop
* `RAUM_ZUSATZBEZEICHNUNG`: drop
* `RAUM_SITZPLAETZE`: rename to `seats`, impute
* `QUADRATMETER`: remame to `area`, impute
* `RAUM_GEBAEUDE_BEREICH_NAME`: rename to `campus`
* `STRASSE`: rename to `address`, convert to longitude & latitude
* `PLZ`: rename to `postal_code`
* `ORT`: rename to `city`

In [68]:
# List of rooms and information about them (number of seats, square meters).
# TU Graz has three campuses: Alte Technik, Neue Technik and Inffeldgasse. The
# mapping of every room to a campus is also stored.

# Information for rooms outside TU Graz premises was missing. Jana Lasser 
# manually searched for and filled in room information for rooms at Uni Graz 
# and added the information to the file /data/raw/Räume.csv. The updated file 
# is stored in /data/cleaned/Räume_cleaned.csv. These rooms are excluded from
# the further analysis anyways though.

df = pd.read_csv(join('../../data/cleaned', 'Räume_cleaned.csv'), 
                    encoding='latin_1')
df = df.rename(columns={
    'RES_NR':'room_id', # unique room id
    'RAUM_SITZPLAETZE':'seats', # number of seats in the room
    'QUADRATMETER':'area', # number of square meters in the room
    'RAUM_GEBAEUDE_BEREICH_NAME':'campus', # campus where the room is located
    'STRASSE':'address',
    'PLZ':'postal_code',
    'ORT':'city',
})
df = df.drop(columns=[
    "RAUM_CODE",
    "RAUM_ZUSATZBEZEICHNUNG"
])
df.to_csv(join(dst, "rooms.csv"), index=False)